In [ ]:
#! python3 -m pip install --upgrade bokeh
#! python3 -m pip install --upgrade jupyter_bokeh


In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
# Plot a complex chart with interactive hover in a few lines of code

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)

group = df.groupby(by=['cyl', 'mfr'])
source = ColumnDataSource(group)

p = figure(width=800, height=300, title="Mean MPG by # Cylinders and Manufacturer",
           x_range=group, toolbar_location=None, tools="")

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Manufacturer grouped by # Cylinders"
p.xaxis.major_label_orientation = 1.2

index_cmap = factor_cmap('cyl_mfr', palette=['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c'], 
                         factors=sorted(df.cyl.unique()), end=1)

p.vbar(x='cyl_mfr', top='mpg_mean', width=1, source=source,
       line_color="white", fill_color=index_cmap, 
       hover_line_color="darkgrey", hover_fill_color=index_cmap)

p.add_tools(HoverTool(tooltips=[("MPG", "@mpg_mean"), ("Cyl, Mfr", "@cyl_mfr")]))

show(p)

In [ ]:
import rbot

In [ ]:
bn = rbot.Market.open("BN", "BTCBUSD")

In [ ]:
ohlcv = bn.ohlcv(rbot.NOW()-rbot.DAYS(10), 0, 60*60)

In [ ]:
ohlcv

In [ ]:
output_notebook()

In [ ]:
inc = ohlcv.open < ohlcv.close
dec = ohlcv.close < ohlcv.open 

In [ ]:
delta = (ohlcv[1:2].index - ohlcv[0:1].index)[0]

In [ ]:
w = delta.total_seconds() * 1_000

In [ ]:
df

In [ ]:
df_inc = df[(df['open'] <= df['close'])]
df_dec = df[(df['close'] < df['open'])] 

In [ ]:
df_inc

In [ ]:
df_dec

In [ ]:
ds = ColumnDataSource(ohlcv)

df_inc = ColumnDataSource(ohlcv[(ohlcv['open'] <= ohlcv['close'])])
df_dec = ColumnDataSource(ohlcv[(ohlcv['close'] < ohlcv['open'])])

delta = (ohlcv[1:2].index - ohlcv[0:1].index)[0]
w = delta.total_seconds() * 1_000 * 0.8

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"


p = figure(x_axis_type="datetime", tools=TOOLS, width=900, height=400,
           title="BTC chart", background_fill_color="#efefef")
   

p.xaxis.major_label_orientation = 0.8 # radians

#p.segment('index', 'high', 'index', 'low', source=ds, color="black")

p.segment('timestamp', 'high', 'timestamp', 'low', source=ds)

vbar_inc = p.vbar('timestamp', w, 'open', 'close', source=df_inc)
vbar_dec = p.vbar('timestamp', w, 'close', 'open', source=df_dec)

#bar_dec = p.vbar(df.index[dec], w, df.open[dec], df.close[dec], color="#eb3c40")
#bar_inc = p.vbar(df.index[inc], w, df.open[inc], df.close[inc], fill_color="white",
#       line_color="#49a3a3", line_width=2)



hover_inc = HoverTool(
    renderers=[vbar_inc],
    tooltips = [
           ("timestamp", "@timestamp{%F %R.%S}"),
           ("open", "@open{0.0}"),
           ("high", "@high{0.0}"),
           ("low", "@low{0.0}"),
           ("close", "@close{0.0}")
    ],
    formatters= {
        "@timestamp": "datetime",
    },
    mode="vline",
    show_arrow=False,
)       
hover_dec = HoverTool(
    renderers=[vbar_dec],
    tooltips = [
           ("timestamp", "@timestamp{%F %R.%S}"),
           ("open", "@open{0.0}"),
           ("high", "@high{0.0}"),
           ("low", "@low{0.0}"),
           ("close", "@close{0.0}")
    ],
    formatters= {
        "@timestamp": "datetime"
    },
    mode="vline",
    show_arrow=False,
)      

p.add_tools(hover_inc)
p.add_tools(hover_dec)

show(p)

In [ ]:


def draw_ohlcv(p, ohlcv):
    ds = ColumnDataSource(ohlcv)

    df_inc = ColumnDataSource(ohlcv[(ohlcv['open'] <= ohlcv['close'])])
    df_dec = ColumnDataSource(ohlcv[(ohlcv['close'] < ohlcv['open'])])

    delta = (ohlcv[1:2].index - ohlcv[0:1].index)[0]
    w = delta.total_seconds() * 1_000 * 0.8



    p.segment('timestamp', 'high', 'timestamp', 'low', source=ds, color="#eb3c40")
    vbar_inc = p.vbar('timestamp', w, 'open', 'close', source=df_inc, fill_color="white", line_color="#49a3a3", line_width=2)
    vbar_dec = p.vbar('timestamp', w, 'close', 'open', source=df_dec)


    hover_inc = HoverTool(
        renderers=[vbar_inc],
        tooltips = [
           ("timestamp", "@timestamp{%F %R.%S}"),
           ("open", "@open{0.0}"),
           ("high", "@high{0.0}"),
           ("low", "@low{0.0}"),
           ("close", "@close{0.0}")
        ],
        formatters= {
            "@timestamp": "datetime",
        },
        mode="vline",
        show_arrow=False,
    )       

    hover_dec = HoverTool(
        renderers=[vbar_dec],
        tooltips = [
           ("timestamp", "@timestamp{%F %R.%S}"),
           ("open", "@open{0.0}"),
           ("high", "@high{0.0}"),
           ("low", "@low{0.0}"),
           ("close", "@close{0.0}")
        ],
        formatters= {
            "@timestamp": "datetime"
        },
        mode="vline",
        show_arrow=False,
    )      

    p.add_tools(hover_inc)
    p.add_tools(hover_dec)
    
    
    
    

In [ ]:
def draw_price_line(p, ohlcv):
    p.line(x=ohlcv.index, y=ohlcv['close'])

In [ ]:
from bokeh.layouts import column 
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure, show
from bokeh.core.properties import Instance


TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

WIDTH=900

dates = np.array(ohlcv.index, dtype=np.datetime64)

p.xaxis.major_label_orientation = 0.8 # radians
p = figure(x_axis_type="datetime", tools=TOOLS, width=WIDTH, height=400,
           title="BTC chart", background_fill_color="#efefef", x_range=(dates[0], dates[-1]))

draw_ohlcv(p, ohlcv)

volume = figure(x_axis_type="datetime", tools=TOOLS, width=WIDTH, height=100,
        title="BTC vol", background_fill_color="#efefef", x_range=p.x_range)

volume.line(x=ohlcv.index, y=ohlcv['volume'])

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                height=130, width=WIDTH, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

draw_price_line(select, ohlcv)

select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, volume, select))



In [ ]:
fig = new_

In [ ]:
p.x_range


In [ ]:
p.x_range

In [ ]:
from collections import OrderedDict

class Chart:
    def __init__(self, width, height, ohlc):
        self.figure = OrderedDict()
        self.width = width
        self.height = height
        self.x_range = None
        self.select = None


        # setup main figure
        dates = np.array(ohlcv.index, dtype=np.datetime64)        
        main = figure(x_axis_type="datetime", tools=TOOLS, width=WIDTH, height=height,
           title="BTC chart", background_fill_color="#efefef", x_range=(dates[0], dates[-1]))
        
        self.x_range = main.x_range
        self.figure['main'] = main
        
        self.draw_ohlcv(main, ohlcv)

        # setup select figure
        select = figure(title="Drag the middle and edges of the selection box to change the range above",
                height= int(height/4), width=WIDTH, y_range=main.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")            

        self.select = select

        self.draw_price_line(select, ohlcv)
        
        range_tool = RangeTool(x_range=self.x_range)
        range_tool.overlay.fill_color = "navy"
        range_tool.overlay.fill_alpha = 0.2

        select.ygrid.grid_line_color = None
        select.add_tools(range_tool)
        select.toolbar.active_multi = range_tool
        

    
    def new_figure(self, name, height, title):
        p = figure(x_axis_type="datetime", width=WIDTH, height=height, tools="", toolbar_location=None,
            title=title, background_fill_color="#efefef", x_range=self.x_range)
        self.figure[name] = p 
        
    def get_figure(self, name):
        return self.figure[name]

    def show(self):
        figure = []
        for key in self.figure:
            print(key)
            figure.append(self.figure[key])

        figure.append(self.select)
            
        show(column(figure))

    def draw_ohlcv(self, p, ohlcv):
        ds = ColumnDataSource(ohlcv)

        df_inc = ColumnDataSource(ohlcv[(ohlcv['open'] <= ohlcv['close'])])
        df_dec = ColumnDataSource(ohlcv[(ohlcv['close'] < ohlcv['open'])])

        delta = (ohlcv[1:2].index - ohlcv[0:1].index)[0]
        w = delta.total_seconds() * 1_000 * 0.8

        p.segment('timestamp', 'high', 'timestamp', 'low', source=ds, color="#eb3c40")
        vbar_inc = p.vbar('timestamp', w, 'open', 'close', source=df_inc, fill_color="white", line_color="#49a3a3", line_width=2)
        vbar_dec = p.vbar('timestamp', w, 'close', 'open', source=df_dec)

        hover_inc = HoverTool(
            renderers=[vbar_inc],
            tooltips = [
                ("timestamp", "@timestamp{%F %R.%S}"),
                ("open", "@open{0.0}"),
                ("high", "@high{0.0}"),
                ("low", "@low{0.0}"),
                ("close", "@close{0.0}")
            ],
            formatters= {
                "@timestamp": "datetime",
            },
            mode="vline",
            show_arrow=False,
        )       

        hover_dec = HoverTool(
            renderers=[vbar_dec],
            tooltips = [
                ("timestamp", "@timestamp{%F %R.%S}"),
                ("open", "@open{0.0}"),
                ("high", "@high{0.0}"),
                ("low", "@low{0.0}"),
                ("close", "@close{0.0}")
            ],
            formatters= {
                "@timestamp": "datetime"
            },
            mode="vline",
            show_arrow=False,
        )      

        p.add_tools(hover_inc)
        p.add_tools(hover_dec)

    def draw_price_line(self, p, ohlc):
        p.line(x=ohlcv.index, y=ohlcv['close'])
        
    def draw_volume_line(self, name, ohlcv):
        p = self.figure[name]
        p.line(x=ohlcv.index, y=ohlcv['volume'])        


In [ ]:
chart = Chart(800, 400, ohlcv)

chart.new_figure('volume', 100, 'trading volume in btc')
chart.draw_volume_line('volume', ohlcv)



In [ ]:
chart.show()

In [ ]:
#https://docs.bokeh.org/en/latest/docs/user_guide/interaction/linking.html


from random import random

from bokeh.layouts import row
from bokeh.models import CrosshairTool, Span
from bokeh.plotting import figure, show

x = [random() * 10 for _ in range(200)]
y = [random() * 10 for _ in range(200)]

width = Span(dimension="width", line_dash="dashed", line_width=2)
height = Span(dimension="height", line_dash="dotted", line_width=2)

p1 = figure(height=400, width=400, x_range=(0, 10), y_range=(0, 10),
            tools="hover", toolbar_location=None)
p1.add_tools(CrosshairTool(overlay=[width, height]))
p1.circle(x, y, radius=0.2, alpha=0.3, hover_alpha=1.0)

p2 = figure(height=400, width=250, x_range=(0, 10), y_range=(0, 10),
            tools="hover", toolbar_location=None)
p2.add_tools(CrosshairTool(overlay=[width, height]))
p2.circle(x, y, radius=0.2, alpha=0.3, hover_alpha=1.0)

show(row(p1, p2))